In [1]:
import sqlalchemy
import pandas as pd
import sqlite3 as sql
import numpy as np

import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import pickle

from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV

In [2]:
database = 'Datasets/diamonds_train.db'
connection = sql.connect(database)

In [3]:
query = '''SELECT tr.index_id, c.clarity, cl.color, ct.cut, dm.depth, dm.'table', dm.x, dm.y, dm.z, tr.price, ci.city, tr.carat
FROM diamonds_properties pr
   INNER JOIN diamonds_clarity c ON pr.clarity_id = c.clarity_id 
   INNER JOIN diamonds_color cl on pr.color_id = cl.color_id 
   INNER JOIN diamonds_cut ct on pr.cut_id = ct.cut_id 
   INNER JOIN diamonds_dimensions dm on pr.index_id = dm.index_id
   INNER JOIN diamonds_transactional tr on pr.index_id = tr.index_id
   INNER JOIN diamonds_city ci on tr.city_id = ci.city_id'''

In [4]:
diamonds = pd.read_sql_query(query,connection)
diamonds

,index_id,clarity,color,cut,depth,table,x,y,z,price,city,carat
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,VS2,J,Premium,62.4,58.0,6.83,6.79,4.25,4268,Dubai,1.21
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,VS2,H,Very Good,63.0,57.0,4.35,4.38,2.75,505,Kimberly,0.32
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,VS1,G,Fair,65.5,55.0,5.62,5.53,3.65,2686,Las Vegas,0.71
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,SI1,D,Good,63.8,56.0,4.68,4.72,3.00,738,Kimberly,0.41
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,SI1,G,Ideal,60.5,59.0,6.55,6.51,3.95,4882,Dubai,1.02
...,...,...,...,...,...,...,...,...,...,...,...,...
40450,f0bc79169405ebeb24e308055156b946ffd819db9b4f75...,VS1,G,Ideal,62.7,57.0,7.10,7.04,4.43,10070,Antwerp,1.34
40451,339916a23bf22b052b54cb2a9b36ee8418c1c68b46acad...,SI2,F,Good,57.1,60.0,8.31,8.25,4.73,12615,Madrid,2.02
40452,46957922b99954654c1deb8d854c3f069bf118b2ce9415...,SI1,H,Ideal,62.7,56.0,6.37,6.42,4.01,5457,Kimberly,1.01
40453,9d733392d362d5c6f1d9b9659b601c7d4b5a1c1c8df579...,VS1,J,Ideal,61.9,54.3,4.45,4.47,2.76,456,Kimberly,0.33


In [5]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.20)
    q3 = df_in[col_name].quantile(0.70)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

In [6]:
remove_outlier(diamonds,"price")

,index_id,clarity,color,cut,depth,table,x,y,z,price,city,carat
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,VS2,J,Premium,62.4,58.0,6.83,6.79,4.25,4268,Dubai,1.21
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,VS2,H,Very Good,63.0,57.0,4.35,4.38,2.75,505,Kimberly,0.32
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,VS1,G,Fair,65.5,55.0,5.62,5.53,3.65,2686,Las Vegas,0.71
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,SI1,D,Good,63.8,56.0,4.68,4.72,3.00,738,Kimberly,0.41
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,SI1,G,Ideal,60.5,59.0,6.55,6.51,3.95,4882,Dubai,1.02
...,...,...,...,...,...,...,...,...,...,...,...,...
40449,1726e8e229489b353e6007fac1dfffb5300a96bf6166cf...,VS1,F,Ideal,61.9,56.0,5.69,5.72,3.53,3710,Kimberly,0.71
40450,f0bc79169405ebeb24e308055156b946ffd819db9b4f75...,VS1,G,Ideal,62.7,57.0,7.10,7.04,4.43,10070,Antwerp,1.34
40452,46957922b99954654c1deb8d854c3f069bf118b2ce9415...,SI1,H,Ideal,62.7,56.0,6.37,6.42,4.01,5457,Kimberly,1.01
40453,9d733392d362d5c6f1d9b9659b601c7d4b5a1c1c8df579...,VS1,J,Ideal,61.9,54.3,4.45,4.47,2.76,456,Kimberly,0.33


In [7]:
diamonds1 = diamonds.to_csv("Datasets/diamonds_train.csv")

In [8]:
diamonds_test = pd.read_csv("Datasets/diamonds_test.csv")

In [9]:
diamonds_test

,id,carat,cut,color,clarity,depth,table,x,y,z,city
0,0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam
...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam
13481,13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City
13482,13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv
13483,13483,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,3.45,Surat


In [10]:
target = "price"
cat_features = ["cut", "color", "clarity", "city"]
num_features = ["carat","depth", "table", "x", "y", "z"]

In [11]:
for i in cat_features:
    diamonds[i] = diamonds[i].astype("category")
    diamonds_test[i] = diamonds_test[i].astype("category")

In [12]:
cat_df = pd.get_dummies(diamonds[cat_features])
num_df = diamonds.loc[:, num_features]
train_df = pd.concat([cat_df, num_df], axis = 1)

cat_df = pd.get_dummies(diamonds_test[cat_features])
num_df = diamonds_test.loc[:, num_features]
test_df = pd.concat([cat_df, num_df], axis = 1)

features = list(cat_df.columns) + list(num_df.columns)

In [13]:
scaler = RobustScaler()
y = diamonds[target]
X = scaler.fit_transform(train_df.loc[:, features].values)

In [14]:
X_test = scaler.fit_transform(test_df.loc[:, features].values)

In [15]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"X_train: {X_train.shape}, X_test: {X_validation.shape}, y_train: {y_train.shape}, y_test: {y_validation.shape}")

X_train: (32364, 39), X_test: (8091, 39), y_train: (32364,), y_test: (8091,)


In [16]:
regressor = RandomForestRegressor()
hyperparameters = regressor.get_params()
regressor.fit(X=X_train, y=y_train)
y_pred = regressor.predict(X_validation)

print('Model:', regressor, '\n')
print('Model hyperparameters:', hyperparameters, '\n')
print('Ground truth target:', y, '\n')
print('Predicted target:', y_pred, '\n')

Model: RandomForestRegressor() 

Model hyperparameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False} 

Ground truth target: 0         4268
1          505
2         2686
3          738
4         4882
         ...  
40450    10070
40451    12615
40452     5457
40453      456
40454     6232
Name: price, Length: 40455, dtype: int64 

Predicted target: [2779.31 2312.27  867.78 ... 2823.92 3833.32 7739.08] 



In [17]:
mean_squared_error(y_validation, y_pred)

314259.7534223856

In [18]:
# RMSE calculation

rmse = mean_squared_error(y_validation, y_pred)**0.5
rmse

560.5887560613266

In [19]:
# R2 calculation

r2 = r2_score(y_validation, y_pred)
r2

0.9807022600286668

In [20]:
y_test = regressor.predict(X_test)

In [21]:
submission = pd.DataFrame({'id': diamonds_test['id'], 'price': y_test})
submission.to_csv("submission21.csv", index = False)